In [ ]:
import pandas as pd
import numpy as np
import sys
from numpy import array
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

The above statements imports all necessary libraries to be used

In [ ]:
from google.colab import files
uploads = files.upload()

Saving train_data.csv to train_data (14).csv


Importing the Train data

In [ ]:
data = pd.read_csv('train_data.csv')
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


Confirm that the data is imported

Deal with NULL Values.


In [ ]:

mean = data['LotFrontage'].mean()
new_data=data.copy(deep=True)

The alley column we replaced all NA values with "Pave"

In [ ]:
missing_values=['NA']
new_data.replace({"NA":np.nan},inplace=True)
new_data.head(n=10)
new_data['LotFrontage'].fillna(int(mean),inplace=True)
#new_data['LotFrontage'].head(10)

#Dealing with Pave
new_data['Alley'].fillna("Pave",inplace=True)
new_data['Alley'].head(10)

0    Pave
1    Pave
2    Pave
3    Pave
4    Pave
5    Pave
6    Pave
7    Pave
8    Pave
9    Pave
Name: Alley, dtype: object

Find other Null values in the data set.

And replace the null values with existing column values


We replaced The PoolQC with actual values because of the Conflicting NA cell data with numpy's NaN value field

In [ ]:
#new_data.isna()
new_data['MiscFeature'].fillna("Shed",inplace=True)
new_data['Fence'].fillna("MnPrv",inplace=True)
#Pool Quality is ordinal data. So we will replace it with actual values
new_data['PoolQC'].fillna(1,inplace=True)
new_data['PoolQC'].replace({"EX":5},inplace=True)
new_data['PoolQC'].replace({"Gd":4},inplace=True)
new_data['PoolQC'].replace({"TA":3},inplace=True)
new_data['PoolQC'].replace({"Fa":2},inplace=True)
#new_data['PoolQC'].head(20)


Check if there is any data remaining which has null values

In [ ]:
new_data.isna().sum()

Id               0
MSSubClass       0
MSZoning         0
LotFrontage      0
LotArea          0
                ..
MoSold           0
YrSold           0
SaleType         0
SaleCondition    0
SalePrice        0
Length: 81, dtype: int64

**Define** the Nominal and Ordinal Data

In [ ]:
nominal=['Street','Alley','LotConfig','Condition1','Condition2',
'RoofStyle','RoofMatl','Exterior1st',	'Exterior2nd','MasVnrType','Foundation','Heating','CentralAir',
'Electrical',
'SaleCondition','SaleType','MiscFeature','PavedDrive','GarageType','Functional']


ordinal=['LotShape',	'LandContour',	'Utilities', 'LandSlope',	'BldgType',	'HouseStyle',		
	'RoofStyle',	'ExterQual',	'ExterCond',	'BsmtQual',	'BsmtCond',	'BsmtExposure',	
'KitchenQual',		'BsmtFinType2',	'BsmtFinType1', 'HeatingQC', 'YearBuilt','YearRemodAdd',
'FireplaceQu'	,	'GarageFinish',		'GarageQual',	'GarageCond',		
'Fence','Neighborhood','MSZoning','MoSold','YrSold','GarageYrBlt']

numerical=['LotFrontage','LotArea',	'BsmtFinSF1',		'BsmtFinSF2',	'BsmtUnfSF',	'TotalBsmtSF',		'1stFlrSF',	
'2ndFlrSF',	'LowQualFinSF',	'GrLivArea',	'BedroomAbvGr',	'KitchenAbvGr','GarageCars',	'GarageArea','PoolArea',
'MasVnrArea','TotRmsAbvGrd','OverallQual',	'OverallCond','Fireplaces','WoodDeckSF','BsmtFullBath','BsmtHalfBath','FullBath','HalfBath',
'OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch','MiscVal','PoolQC','MSSubClass',]

target=['SalePrice']

#confirm all features are selected
print(len(nominal)+len(ordinal)+len(numerical))
new_data[nominal]


80


,Street,Alley,LotConfig,Condition1,Condition2,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,Foundation,Heating,CentralAir,Electrical,SaleCondition,SaleType,MiscFeature,PavedDrive,GarageType,Functional
0,Pave,Pave,Inside,Norm,Norm,Gable,CompShg,VinylSd,VinylSd,BrkFace,PConc,GasA,Y,SBrkr,Normal,WD,Shed,Y,Attchd,Typ
1,Pave,Pave,FR2,Feedr,Norm,Gable,CompShg,MetalSd,MetalSd,None,CBlock,GasA,Y,SBrkr,Normal,WD,Shed,Y,Attchd,Typ
2,Pave,Pave,Inside,Norm,Norm,Gable,CompShg,VinylSd,VinylSd,BrkFace,PConc,GasA,Y,SBrkr,Normal,WD,Shed,Y,Attchd,Typ
3,Pave,Pave,Corner,Norm,Norm,Gable,CompShg,Wd Sdng,Wd Shng,None,BrkTil,GasA,Y,SBrkr,Abnorml,WD,Shed,Y,Detchd,Typ
4,Pave,Pave,FR2,Norm,Norm,Gable,CompShg,VinylSd,VinylSd,BrkFace,PConc,GasA,Y,SBrkr,Normal,WD,Shed,Y,Attchd,Typ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,Pave,Pave,Inside,Norm,Norm,Gable,CompShg,VinylSd,VinylSd,None,PConc,GasA,Y,SBrkr,Normal,WD,Shed,Y,Attchd,Typ
1456,Pave,Pave,Inside,Norm,Norm,Gable,CompShg,Plywood,Plywood,Stone,CBlock,GasA,Y,SBrkr,Normal,WD,Shed,Y,Attchd,Min1
1457,Pave,Pave,Inside,Norm,Norm,Gable,CompShg,CemntBd,CmentBd,None,Stone,GasA,Y,SBrkr,Normal,WD,Shed,Y,Attchd,Typ
1458,Pave,Pave,Inside,Norm,Norm,Hip,CompShg,MetalSd,MetalSd,None,CBlock,GasA,Y,FuseA,Normal,WD,Shed,Y,Attchd,Typ


Define categorical data to prepare for one hot encoding

Converting years to type string

In [ ]:
categorical_data=new_data.copy(deep=True)
column_names=list(categorical_data.columns)

for feature in column_names:
  categorical_data[feature]=categorical_data[feature].astype('category')

for feature in ordinal:
  categorical_data[feature]= categorical_data[feature].cat.codes

categorical_data

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,Pave,3,3,0,Inside,0,5,Norm,Norm,0,5,7,5,104,53,1,CompShg,VinylSd,VinylSd,BrkFace,196.0,2,4,PConc,Gd,3,3,2,706,5,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,2,8,Typ,0,-1,Attchd,2003.0,1,2,548,4,4,Y,0,61,0,0,0,0,1,2,Shed,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,Pave,3,3,0,FR2,0,24,Feedr,Norm,0,2,6,8,77,26,1,CompShg,MetalSd,MetalSd,None,0.0,3,4,CBlock,Gd,3,1,0,978,5,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,3,6,Typ,1,4,Attchd,1976.0,1,2,460,4,4,Y,298,0,0,0,0,0,1,2,Shed,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,Pave,0,3,0,Inside,0,5,Norm,Norm,0,5,7,5,102,52,1,CompShg,VinylSd,VinylSd,BrkFace,162.0,2,4,PConc,Gd,3,2,2,486,5,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,2,6,Typ,1,4,Attchd,2001.0,1,2,608,4,4,Y,0,42,0,0,0,0,1,2,Shed,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,Pave,0,3,0,Corner,0,6,Norm,Norm,0,5,7,5,19,20,1,CompShg,Wd Sdng,Wd Shng,None,0.0,3,4,BrkTil,TA,1,3,0,216,5,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,2,7,Typ,1,2,Detchd,1998.0,2,3,642,4,4,Y,0,35,272,0,0,0,1,2,Shed,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,Pave,0,3,0,FR2,0,15,Norm,Norm,0,5,8,5,101,50,1,CompShg,VinylSd,VinylSd,BrkFace,350.0,2,4,PConc,Gd,3,0,2,655,5,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,2,9,Typ,1,4,Attchd,2000.0,1,3,836,4,4,Y,192,84,0,0,0,0,1,2,Shed,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,Pave,3,3,0,Inside,0,8,Norm,Norm,0,5,6,5,100,50,1,CompShg,VinylSd,VinylSd,None,0.0,3,4,PConc,Gd,3,3,5,0,5,0,953,953,GasA,...,Y,SBrkr,953,694,0,1647,0,0,2,1,3,1,3,7,Typ,1,4,Attchd,1999.0,1,2,460,4,4,Y,0,40,0,0,0,0,1,2,Shed,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,Pave,3,3,0,Inside,0,14,Norm,Norm,0,2,6,6,79,38,1,CompShg,Plywood,Plywood,Stone,119.0,3,4,CBlock,Gd,3,3,0,790,4,163,589,1542,GasA,...,Y,SBrkr,2073,0,0,2073,1,0,2,0,3,1,3,7,Min1,2,4,Attchd,1978.0,2,2,500,4,4,Y,349,0,0,0,0,0,1,2,Shed,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,Pave,3,3,0,Inside,0,6,Norm,Norm,0,5,7,9,44,56,1,CompShg,CemntBd,CmentBd,None,0.0,0,2,Stone,TA,1,3,2,275,5,0,877,1152,GasA,...,Y,SBrkr,1188,1152,0,2340,0,0,2,0,4,1,2,9,Typ,2,2,Attchd,1941.0,1,1,252,4,4,Y,0,60,0,0,0,0,1,0,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,Pave,3,3,0,Inside,0,12,Norm,Norm,0,2,5,6,51,46,3,CompShg,MetalSd,MetalSd,None,0.0,3,4,CBlock,TA,3,2,2,49,4,1029,0,1078,GasA,...,Y,FuseA,1078,0,0,1078,1,0,1,0,2,1,2,5,Typ,0,-1,Attchd,1950.0,2,1,240,4,4,Y,366,0,112,0,0,0,1,2,Shed,0,4,2010,WD,Normal,142125


Define the method for One hot encoding for ordinal Data

In [ ]:
data_copy=categorical_data.copy()
data_copy=pd.concat([pd.get_dummies(data_copy[nominal]),data_copy],axis=1)
#Onehot Encoding
#Problem --- Increases dimensionality
data_copy.drop(nominal,axis=1,inplace=True)
data_copy



,Street_Grvl,Street_Pave,Alley_Grvl,Alley_Pave,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,Condition1_Artery,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Artery,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,RoofMatl_ClyTile,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsbShng,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,...,PoolArea,PoolQC,Fence,MiscVal,MoSold,YrSold,SalePrice,LotFrontage,LotArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BedroomAbvGr,KitchenAbvGr,GarageCars,GarageArea,PoolArea,MasVnrArea,TotRmsAbvGrd,OverallQual,OverallCond,Fireplaces,WoodDeckSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,MiscVal,PoolQC,MSSubClass,SalePrice
0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,2,0,2,2008,208500,65.0,8450,706,0,150,856,856,854,0,1710,3,1,2,548,0,196.0,8,7,5,0,0,1,0,2,1,61,0,0,0,0,1,60,208500
1,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,2,0,5,2007,181500,80.0,9600,978,0,284,1262,1262,0,0,1262,3,1,2,460,0,0.0,6,6,8,1,298,0,1,2,0,0,0,0,0,0,1,20,181500
2,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,2,0,9,2008,223500,68.0,11250,486,0,434,920,920,866,0,1786,3,1,2,608,0,162.0,6,7,5,1,0,1,0,2,1,42,0,0,0,0,1,60,223500
3,0,1,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,2,0,2,2006,140000,60.0,9550,216,0,540,756,961,756,0,1717,3,1,3,642,0,0.0,7,7,5,1,0,1,0,1,0,35,272,0,0,0,1,70,140000
4,0,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,2,0,12,2008,250000,84.0,14260,655,0,490,1145,1145,1053,0,2198,4,1,3,836,0,350.0,9,8,5,1,192,1,0,2,1,84,0,0,0,0,1,60,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,2,0,8,2007,175000,62.0,7917,0,0,953,953,953,694,0,1647,3,1,2,460,0,0.0,7,6,5,1,0,0,0,2,1,40,0,0,0,0,1,60,175000
1456,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,2,0,2,2010,210000,85.0,13175,790,163,589,1542,2073,0,0,2073,3,1,2,500,0,119.0,7,6,6,2,349,1,0,2,0,0,0,0,0,0,1,20,210000
1457,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,...,0,1,0,2500,5,2010,266500,66.0,9042,275,0,877,1152,1188,1152,0,2340,4,1,1,252,0,0.0,9,7,9,2,0,0,0,2,0,60,0,0,0,2500,1,70,266500
1458,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,2,0,4,2010,142125,68.0,9717,49,1029,0,1078,1078,0,0,1078,2,1,1,240,0,0.0,5,5,6,0,366,1,0,1,0,0,112,0,0,0,1,20,142125


In [ ]:
new_data1 = pd.concat([data_copy,new_data[numerical],new_data[target]],axis=1)
new_data1

,Street_Grvl,Street_Pave,Alley_Grvl,Alley_Pave,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,Condition1_Artery,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Artery,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,RoofMatl_ClyTile,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsbShng,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,...,PoolArea,PoolQC,Fence,MiscVal,MoSold,YrSold,SalePrice,LotFrontage,LotArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BedroomAbvGr,KitchenAbvGr,GarageCars,GarageArea,PoolArea,MasVnrArea,TotRmsAbvGrd,OverallQual,OverallCond,Fireplaces,WoodDeckSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,MiscVal,PoolQC,MSSubClass,SalePrice
0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,2,0,2,2008,208500,65.0,8450,706,0,150,856,856,854,0,1710,3,1,2,548,0,196.0,8,7,5,0,0,1,0,2,1,61,0,0,0,0,1,60,208500
1,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,2,0,5,2007,181500,80.0,9600,978,0,284,1262,1262,0,0,1262,3,1,2,460,0,0.0,6,6,8,1,298,0,1,2,0,0,0,0,0,0,1,20,181500
2,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,2,0,9,2008,223500,68.0,11250,486,0,434,920,920,866,0,1786,3,1,2,608,0,162.0,6,7,5,1,0,1,0,2,1,42,0,0,0,0,1,60,223500
3,0,1,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,2,0,2,2006,140000,60.0,9550,216,0,540,756,961,756,0,1717,3,1,3,642,0,0.0,7,7,5,1,0,1,0,1,0,35,272,0,0,0,1,70,140000
4,0,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,2,0,12,2008,250000,84.0,14260,655,0,490,1145,1145,1053,0,2198,4,1,3,836,0,350.0,9,8,5,1,192,1,0,2,1,84,0,0,0,0,1,60,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,2,0,8,2007,175000,62.0,7917,0,0,953,953,953,694,0,1647,3,1,2,460,0,0.0,7,6,5,1,0,0,0,2,1,40,0,0,0,0,1,60,175000
1456,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,2,0,2,2010,210000,85.0,13175,790,163,589,1542,2073,0,0,2073,3,1,2,500,0,119.0,7,6,6,2,349,1,0,2,0,0,0,0,0,0,1,20,210000
1457,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,...,0,1,0,2500,5,2010,266500,66.0,9042,275,0,877,1152,1188,1152,0,2340,4,1,1,252,0,0.0,9,7,9,2,0,0,0,2,0,60,0,0,0,2500,1,70,266500
1458,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,2,0,4,2010,142125,68.0,9717,49,1029,0,1078,1078,0,0,1078,2,1,1,240,0,0.0,5,5,6,0,366,1,0,1,0,0,112,0,0,0,1,20,142125


In [ ]:
#Slicing the data
all_features = list(data_copy.columns)
X = data_copy.loc[:,all_features].values
y = new_data1.loc[:,['SalePrice']].values
all_features

['Street_Grvl',
 'Street_Pave',
 'Alley_Grvl',
 'Alley_Pave',
 'LotConfig_Corner',
 'LotConfig_CulDSac',
 'LotConfig_FR2',
 'LotConfig_FR3',
 'LotConfig_Inside',
 'Condition1_Artery',
 'Condition1_Feedr',
 'Condition1_Norm',
 'Condition1_PosA',
 'Condition1_PosN',
 'Condition1_RRAe',
 'Condition1_RRAn',
 'Condition1_RRNe',
 'Condition1_RRNn',
 'Condition2_Artery',
 'Condition2_Feedr',
 'Condition2_Norm',
 'Condition2_PosA',
 'Condition2_PosN',
 'Condition2_RRAe',
 'Condition2_RRAn',
 'Condition2_RRNn',
 'RoofMatl_ClyTile',
 'RoofMatl_CompShg',
 'RoofMatl_Membran',
 'RoofMatl_Metal',
 'RoofMatl_Roll',
 'RoofMatl_Tar&Grv',
 'RoofMatl_WdShake',
 'RoofMatl_WdShngl',
 'Exterior1st_AsbShng',
 'Exterior1st_AsphShn',
 'Exterior1st_BrkComm',
 'Exterior1st_BrkFace',
 'Exterior1st_CBlock',
 'Exterior1st_CemntBd',
 'Exterior1st_HdBoard',
 'Exterior1st_ImStucc',
 'Exterior1st_MetalSd',
 'Exterior1st_Plywood',
 'Exterior1st_Stone',
 'Exterior1st_Stucco',
 'Exterior1st_VinylSd',
 'Exterior1st_Wd Sdng

In [ ]:
X


array([[0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 60, 'RL', 65.0, 8450,
        3, 3, 0, 0, 5, 0, 5, 7, 5, 104, 53, 196.0, 2, 4, 'Gd', 3, 3, 2,
        706, 5, 0, 150, 856, 0, 856, 854, 0, 1710, 1, 0, 2, 1, 3, 1, 2,
        8, 0, -1, 2003.0, 1, 2, 548, 4, 4, 0, 61, 0, 0, 0, 0, 1, 2, 0, 2,
        2008, 208500],
       [0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 1

In [ ]:
y

array([[208500, 208500],
       [181500, 181500],
       [223500, 223500],
       [140000, 140000],
       [250000, 250000],
       [143000, 143000],
       [307000, 307000],
       [200000, 200000],
       [129900, 129900],
       [118000, 118000],
       [129500, 129500],
       [345000, 345000],
       [144000, 144000],
       [279500, 279500],
       [157000, 157000],
       [132000, 132000],
       [149000, 149000],
       [90000, 90000],
       [159000, 159000],
       [139000, 139000],
       [325300, 325300],
       [139400, 139400],
       [230000, 230000],
       [129900, 129900],
       [154000, 154000],
       [256300, 256300],
       [134800, 134800],
       [306000, 306000],
       [207500, 207500],
       [68500, 68500],
       [40000, 40000],
       [149350, 149350],
       [179900, 179900],
       [165500, 165500],
       [277500, 277500],
       [309000, 309000],
       [145000, 145000],
       [153000, 153000],
       [109000, 109000],
       [82000, 82000],
       [

The next Block will prepare the data and slice the sale price

We also found our new array from the following block of code

In [ ]:
#Standardize X
X = StandardScaler().fit_transform(X)
X


ValueError: ignored

Our Code did not work here, but the plotting code for the 3D is as shown below

In [ ]:
#Principle components
pca=PCA(n_components=2)
pc_s=pca.fit_transform(X)
#Returning Principle Components
pca_data=pd.DataFrame(pc_s,columns=['PC1','PC2'])
final_data = pd.concat([pca_data,data[target]],axis=1)
final_data

In [ ]:
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
ax = plt.axes(projection='3d')

# Data for a three-dimensional line
zline = data[target]
xline = pca_data['PC1']
yline = pca_data['PC2']
ax.plot3D(xline, yline, zline, 'gray')